---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    train.csv - the training set (all tickets issued 2004-2011)
    test.csv - the test set (all tickets issued 2012-2016)
    addresses.csv & latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32

In [2]:
import pandas as pd
import numpy as np

train=pd.read_csv('train.csv', encoding = 'ISO-8859-1')

train.head()

/opt/conda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,ticket_id,agency_name,inspector_name,violator_name,violation_street_number,violation_street_name,violation_zip_code,mailing_address_str_number,mailing_address_str_name,city,...,clean_up_cost,judgment_amount,payment_amount,balance_due,payment_date,payment_status,collection_status,grafitti_status,compliance_detail,compliance
0,22056,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","INVESTMENT INC., MIDWEST MORTGAGE",2900.0,TYLER,NaN,3.0,S. WICKER,CHICAGO,...,0.0,305.0,0.0,305.0,NaN,NO PAYMENT APPLIED,NaN,NaN,non-compliant by no payment,0.0
1,27586,"Buildings, Safety Engineering & Env Department","Williams, Darrin","Michigan, Covenant House",4311.0,CENTRAL,NaN,2959.0,Martin Luther King,Detroit,...,0.0,855.0,780.0,75.0,2005-06-02 00:00:00,PAID IN FULL,NaN,NaN,compliant by late payment within 1 month,1.0
2,22062,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","SANDERS, DERRON",1449.0,LONGFELLOW,NaN,23658.0,P.O. BOX,DETROIT,...,0.0,0.0,0.0,0.0,NaN,NO PAYMENT APPLIED,NaN,NaN,not responsible by disposition,NaN
3,22084,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","MOROSI, MIKE",1441.0,LONGFELLOW,NaN,5.0,ST. CLAIR,DETROIT,...,0.0,0.0,0.0,0.0,NaN,NO PAYMENT APPLIED,NaN,NaN,not responsible by disposition,NaN
4,22093,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","NATHANIEL, NEAL",2449.0,CHURCHILL,NaN,7449.0,CHURCHILL,DETROIT,...,0.0,0.0,0.0,0.0,NaN,NO PAYMENT APPLIED,NaN,NaN,not responsible by disposition,NaN


In [3]:
test=pd.read_csv('test.csv')
test.head()

,ticket_id,agency_name,inspector_name,violator_name,violation_street_number,violation_street_name,violation_zip_code,mailing_address_str_number,mailing_address_str_name,city,...,violation_description,disposition,fine_amount,admin_fee,state_fee,late_fee,discount_amount,clean_up_cost,judgment_amount,grafitti_status
0,284932,Department of Public Works,"Granberry, Aisha B","FLUELLEN, JOHN A",10041.0,ROSEBERRY,NaN,141,ROSEBERRY,DETROIT,...,Failure to secure City or Private solid waste ...,Responsible by Default,200.0,20.0,10.0,20.0,0.0,0.0,250.0,NaN
1,285362,Department of Public Works,"Lusk, Gertrina","WHIGHAM, THELMA",18520.0,EVERGREEN,NaN,19136,GLASTONBURY,DETROIT,...,Allowing bulk solid waste to lie or accumulate...,Responsible by Default,1000.0,20.0,10.0,100.0,0.0,0.0,1130.0,NaN
2,285361,Department of Public Works,"Lusk, Gertrina","WHIGHAM, THELMA",18520.0,EVERGREEN,NaN,19136,GLASTONBURY,DETROIT,...,Improper placement of Courville container betw...,Responsible by Default,100.0,20.0,10.0,10.0,0.0,0.0,140.0,NaN
3,285338,Department of Public Works,"Talbert, Reginald","HARABEDIEN, POPKIN",1835.0,CENTRAL,NaN,2246,NELSON,WOODHAVEN,...,Allowing bulk solid waste to lie or accumulate...,Responsible by Default,200.0,20.0,10.0,20.0,0.0,0.0,250.0,NaN
4,285346,Department of Public Works,"Talbert, Reginald","CORBELL, STANLEY",1700.0,CENTRAL,NaN,3435,MUNGER,LIVONIA,...,Violation of time limit for approved container...,Responsible by Default,100.0,20.0,10.0,10.0,0.0,0.0,140.0,NaN


In [4]:
adress=pd.read_csv('addresses.csv')
latlons = pd.read_csv('latlons.csv')
adress.tail()

,ticket_id,address
311302,376496,"12032 santa rosa, Detroit MI 48204"
311303,376497,"12032 santa rosa, Detroit MI 48204"
311304,376499,"11832 kilbourne, Detroit MI 48213"
311305,376500,"11848 kilbourne, Detroit MI 48213"
311306,369851,"6100 ironwood, Detroit MI 48210"


In [5]:
latlons.head()

,address,lat,lon
0,"4300 rosa parks blvd, Detroit MI 48208",42.346169,-83.079962
1,"14512 sussex, Detroit MI",42.394657,-83.194265
2,"3456 garland, Detroit MI",42.373779,-82.986228
3,"5787 wayburn, Detroit MI",42.403342,-82.957805
4,"5766 haverhill, Detroit MI",42.407255,-82.946295


In [6]:
address=pd.merge(adress, latlons, how='left', left_on='address', right_on='address')
address.head()

,ticket_id,address,lat,lon
0,22056,"2900 tyler, Detroit MI",42.390729,-83.124268
1,27586,"4311 central, Detroit MI",42.326937,-83.135118
2,22062,"1449 longfellow, Detroit MI",42.380516,-83.096069
3,22084,"1441 longfellow, Detroit MI",42.380570,-83.095919
4,22093,"2449 churchill, Detroit MI",42.145257,-83.208233


In [7]:
print(address.shape, train.shape, test.shape)

(311307, 4) (250306, 34) (61001, 27)


In [8]:
#how 
train['ticket_issued_date'].isnull().value_counts()

False    250306
Name: ticket_issued_date, dtype: int64

In [9]:
train['hearing_date'].isnull().value_counts()

False    237815
True      12491
Name: hearing_date, dtype: int64

In [26]:
def blight_model():

    import pandas as pd
    import numpy as np
    from sklearn.neural_network import MLPClassifier
    from sklearn.preprocessing import MinMaxScaler
    from datetime import datetime
    
    #train = pd.read_csv('train.csv', encoding = 'ISO-8859-1', low_memory=False)
    train=pd.read_csv('train.csv', dtype={'zip_code':str, 'non_us_str_code':str, 'grafitti_status':str}, encoding = 'ISO-8859-1')
    test = pd.read_csv('test.csv')
    address =  pd.read_csv('addresses.csv')
    latlons = pd.read_csv('latlons.csv')

    def overdue(a, b):
        
        if not a or type(a)!=str: 
            return 73
        hearing_date = datetime.strptime(a, "%Y-%m-%d %H:%M:%S")
        ticket_issued_date = datetime.strptime(b, "%Y-%m-%d %H:%M:%S")
        over= hearing_date - ticket_issued_date
        return over.days

    address=pd.merge(address, latlons, how='left', left_on='address', right_on='address')
    train=pd.merge(train, address, how='inner', left_on=['ticket_id'], right_on=['ticket_id'])
    test=pd.merge(test, address, how='inner', left_on=['ticket_id'], right_on=['ticket_id'])


    #not very efficient way, for now...
    train['overdues'] = train.apply(lambda x: overdue(x['hearing_date'], x['ticket_issued_date']), axis=1)
    test['overdues'] = test.apply(lambda x: overdue(x['hearing_date'], x['ticket_issued_date']), axis=1)

    #  Not responsible = reason unknown, data unavailable in the test set, let's get rid of it 
    train = train[(train['compliance'] == 0) | (train['compliance'] == 1)]

    train.state.fillna(method='pad', inplace=True)
    test.state.fillna(method='pad', inplace=True)

    feat = ['agency_name', 'state', 'disposition']
    train= pd.get_dummies(train, columns=feat)
    test= pd.get_dummies(test, columns=feat)

    # removing not relevant features
    to_remove=['fine_amount', 'violator_name', 'zip_code', 'country', 'city','inspector_name', 'violation_street_number', 
               'violation_street_name', 'violation_zip_code', 'violation_description', 'mailing_address_str_number', 
               'mailing_address_str_name', 'non_us_str_code', 'ticket_issued_date', 'hearing_date', 'grafitti_status', 
               'violation_code', 'address']
    train.drop(to_remove, axis=1, inplace=True)
    train.drop(['balance_due', 'collection_status','compliance_detail','payment_amount','payment_date','payment_status'], axis=1, inplace=True)
    test.drop(to_remove, axis=1, inplace=True)

    #porzadek z geo
    train.lat.fillna(method='pad', inplace=True)
    train.lon.fillna(method='pad', inplace=True)
    test.lat.fillna(method='pad', inplace=True)
    test.lon.fillna(method='pad', inplace=True)

    features=train.columns.drop('compliance')
    b=test.columns

    features=list(filter(lambda x: x in b, features))       


    X_train = train[features]
    y_train = train.compliance
    X_test = test[features]

    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    clf = MLPClassifier(hidden_layer_sizes = [100, 10], alpha = 5,
                           random_state = 5, solver='lbfgs', verbose=0)
    clf.fit(X_train_scaled, y_train)

    test_pred = clf.predict_proba(X_test_scaled)

    result=pd.DataFrame(test_pred[:,1], index=X_test['ticket_id'])
    
    return result
blight_model()

,0
ticket_id,
284932,0.072473
285362,0.018179
285361,0.076522
285338,0.083579
285346,0.097421
285345,0.083729
285347,0.105001
285342,0.783578
285530,0.006079
